In [1]:
import sys
sys.path.append("../src")
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import lightgbm as lgb
from dataLoad import trainValidLoad
from metric import ndcg_calculator, hit_at_k

path= "../dataset/"
train, train_valid, sample_sumbission = trainValidLoad(path)

--- LgbmRanker + ALS + MP + (Word2Vec) Ensemble      
      
feature :      
*    datset - basic features from featureExtract      
*    Mp - general most popular feature      
*    ALS - user / item factor      
<!-- *    Word2Vec - item segmentation       -->
      
candidate :      
*    Mp  - general/recent weeks Most popular      
<!-- *    Word2Vec - similar items       --> 

In [2]:
# load features

general_MP_feature = pd.read_parquet("../output/general_MP_feature.parquet")
user_factor_als = pd.read_parquet("../output/user_factor_als.parquet")
item_factor_als = pd.read_parquet("../output/item_factor_als.parquet")

# load candidates
recent30days_MP_candidate = pd.read_parquet("../output/recent30days_MP_candidate.parquet")
popular_items_cand = pd.read_parquet("../output/popular_items_cand.parquet")

In [3]:
candidate_1 = recent30days_MP_candidate
candidate_2 = popular_items_cand
candidate = [candidate_1, candidate_2]
cand = pd.concat(candidate)
cand.drop_duplicates(subset=["user","item_id"],inplace=True)

print("cand 데이터 수:",len(cand))
print("cand nunique: ", cand.item_id.nunique())

cand 데이터 수: 119636648
cand nunique:  672116


In [9]:
general_MP_feature  # int, fillna

,item_id,general_counts_ver1,general_counts_ver2
0,145550,5718.0,2409.0
1,15188,5642.0,56.0
2,591848,5474.0,8294.0
3,212441,5224.0,7604.0
4,535897,5022.0,869.0
...,...,...,...
241303,403612,NaN,1.0
241304,403611,NaN,1.0
241305,403609,NaN,1.0
241306,403558,NaN,1.0


In [15]:
# add feature 
candidate = pd.merge(cand, general_MP_feature, how="left", on="item_id")
candidate_add_features = candidate.copy()

# als vector merge
candidate_add_features = pd.merge(candidate_add_features, item_factor_als, how="left", on="item_id")
candidate_add_features = pd.merge(candidate_add_features, user_factor_als, how="left", on="user")

print("cand 피처 수:",len(candidate_add_features))

In [ ]:
# 라벨 부여
def label_create(candidate_add_features_df, label_df):
    train = pd.merge(candidate_add_features_df, label_df, how="left", on=["user","item_id"])
    train["click_num"] = train["click_num"].fillna(0)
    return train

In [ ]:
train = label_create(candidate_add_features_df, label_df)